In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
import math
from torch.utils.data import DataLoader, TensorDataset,RandomSampler
from math import exp, sqrt,pi
import time

import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D

In [4]:

def train(device,x,y,xb,yb,ub,learning_rate,epochs):
	x = torch.Tensor(x).to(device)
	y = torch.Tensor(y).to(device) 
	xb1,xb2,xb3,xb4 = xb
	yb1,yb2,yb3,yb4 = yb
	ub1,ub2,ub3,ub4 = ub
	xb1 = torch.Tensor(xb1).to(device) 	
	yb1 = torch.Tensor(yb1).to(device) 
	ub1 = torch.Tensor(ub1).to(device)
	xb2 = torch.Tensor(xb2).to(device) 	
	yb2 = torch.Tensor(yb2).to(device) 
	ub2 = torch.Tensor(ub2).to(device)
	xb3 = torch.Tensor(xb3).to(device) 	
	yb3 = torch.Tensor(yb3).to(device) 
	ub3 = torch.Tensor(ub3).to(device)
	xb4 = torch.Tensor(xb4).to(device) 	
	yb4 = torch.Tensor(yb4).to(device) 
	ub4 = torch.Tensor(ub4).to(device)

	h_n = 128
	input_n = 2  
	class Swish(nn.Module):
		def __init__(self, inplace=True):
			super(Swish, self).__init__()
			self.inplace = inplace

		def forward(self, x):
			if self.inplace:
				x.mul_(torch.sigmoid(x))
				return x
			else:
				return x * torch.sigmoid(x)
	

	class Net(nn.Module):
		def __init__(self):
			super(Net, self).__init__()
			self.main = nn.Sequential(
				nn.Linear(input_n,h_n),
				Swish(),
				nn.Linear(h_n,h_n),
				Swish(),
				nn.Linear(h_n,h_n),
				Swish(),
				nn.Linear(h_n,h_n),
				#Swish(),
				#nn.Linear(h_n,h_n),
				Swish(),
				nn.Linear(h_n,h_n),
				Swish(),
				nn.Linear(h_n,h_n),
        		Swish(),
				nn.Linear(h_n,1),
			)		
		def forward(self,x):		
			output = self.main(x)
			return  output

	
	net = Net().to(device)
	
	def init_normal(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)

	net.apply(init_normal)

	optimizer2 = optim.Adam(net.parameters(), lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)

	def Loss_criterion(x,y):
		x.requires_grad = True
		y.requires_grad = True
		points = torch.cat((x,y),1) 
		U = net(points)
		U = U.view(len(U),-1)
		U_x = torch.autograd.grad(U,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		U_xx = torch.autograd.grad(U_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		U_y = torch.autograd.grad(U,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		U_yy = torch.autograd.grad(U_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		loss1 = U_xx + U_yy
		return nn.MSELoss()(loss1,torch.zeros_like(loss1)) 

	def Loss_BC(xb1,xb2,xb3,xb4,yb1,yb2,yb3,ub1,ub2,ub3,ub4):
		
		loss_bc = 0

		net1 = torch.cat((xb1, yb1), 1)
		out1 = net(net1)
		ub1_pred = out1.view(len(out1), -1)
		loss_f = nn.MSELoss()
		loss_bc += loss_f(ub1_pred,ub1)
		
		net2 = torch.cat((xb2, yb2), 1)
		out2 = net(net2)
		ub2_pred = out2.view(len(out2), -1)
		loss_f = nn.MSELoss()
		loss_bc += loss_f(ub2_pred,ub2)

		net3 = torch.cat((xb3, yb3), 1)
		out3 = net(net3)
		ub3_pred = out3.view(len(out3), -1)
		loss_f = nn.MSELoss()
		loss_bc += loss_f(ub3_pred,ub3)

		net4 = torch.cat((xb4, yb4), 1)
		out4 = net(net4)
		ub4_pred = out4.view(len(out4), -1)
		loss_f = nn.MSELoss()
		loss_bc += loss_f(ub4_pred,ub4)

		return loss_bc


	
	losses = []
	tic = time.time()


	for epoch in range(epochs):
		net.zero_grad()
		loss_eqn = Loss_criterion(x,y)
		loss_bc = Loss_BC(xb1,xb2,xb3,xb4,yb1,yb2,yb3,ub1,ub2,ub3,ub4)
		loss = loss_eqn + loss_bc
		loss.backward()
			
		optimizer2.step() 
		if epoch % 10 ==0:
			print('Train Epoch: {} \tLoss: {:.10f} \t Loss BC {:.6f}'.format(
				epoch, loss.item(),loss_bc.item()))
			losses.append(loss.item())

	toc = time.time()
	elapseTime = toc - tic
	print ("elapse time in parallel = ", elapseTime)

	net_in = torch.cat((x,y),1)
	output = net(net_in)  #evaluate model
	
	return output 


device = torch.device("cpu")

batchsize = 128 
learning_rate = 1e-5   

epochs = 500 

n = 100 

x = np.linspace(-1.,1.,100 )
y = np.linspace(-1.,1.,100)
x, y = np.meshgrid(x, y)
x = np.reshape(x, (np.size(x[:]),1))
y = np.reshape(y, (np.size(y[:]),1))

xb1 = np.linspace(-1.,1., 100 )
yb1 = np.linspace(-1.,-1., 100 ) 
cb1 = xb1*yb1
xb1 = xb1.reshape(-1, 1)
yb1 = yb1.reshape(-1, 1)
cb1 = cb1.reshape(-1, 1)

xb2 = np.linspace(-1.,1., 100 )
yb2 = np.linspace(1.,1., 100 ) 
cb2 = xb2*yb2
xb2 = xb2.reshape(-1, 1)
yb2 = yb2.reshape(-1, 1)
cb2 = cb2.reshape(-1, 1)

xb3 = np.linspace(1.,1., 100 )
yb3 = np.linspace(-1.,1., 100 ) 
cb3 = xb3*yb3
xb3 = xb3.reshape(-1, 1)
yb3 = yb3.reshape(-1, 1)
cb3 = cb3.reshape(-1, 1)

xb4 = np.linspace(-1.,-1., 100 )
yb4 = np.linspace(-1.,1., 100 ) 
cb4 = xb4*yb4
xb4 = xb4.reshape(-1, 1)
yb4 = yb4.reshape(-1, 1)
cb4 = cb4.reshape(-1, 1)

xb = [xb1,xb2,xb3,xb4]
yb = [yb1,yb2,yb3,yb4]
cb = [cb1,cb2,cb3,cb4]

output = train(device,x,y,xb,yb,cb,learning_rate,epochs)



Train Epoch: 0 	Loss: 7.5702314377 	 Loss BC 1.765891
Train Epoch: 10 	Loss: 6.1825733185 	 Loss BC 1.571737
Train Epoch: 20 	Loss: 5.0387949944 	 Loss BC 1.410690
Train Epoch: 30 	Loss: 4.1195440292 	 Loss BC 1.278707
Train Epoch: 40 	Loss: 3.3917131424 	 Loss BC 1.170317
Train Epoch: 50 	Loss: 2.8187460899 	 Loss BC 1.079923
Train Epoch: 60 	Loss: 2.3673033714 	 Loss BC 1.002657
Train Epoch: 70 	Loss: 2.0098509789 	 Loss BC 0.934665
Train Epoch: 80 	Loss: 1.7247533798 	 Loss BC 0.873084
Train Epoch: 90 	Loss: 1.4953708649 	 Loss BC 0.815898
Train Epoch: 100 	Loss: 1.3089921474 	 Loss BC 0.761759
Train Epoch: 110 	Loss: 1.1558911800 	 Loss BC 0.709824
Train Epoch: 120 	Loss: 1.0286021233 	 Loss BC 0.659624
Train Epoch: 130 	Loss: 0.9213781357 	 Loss BC 0.610951
Train Epoch: 140 	Loss: 0.8297936916 	 Loss BC 0.563766
Train Epoch: 150 	Loss: 0.7504483461 	 Loss BC 0.518142
Train Epoch: 160 	Loss: 0.6807453036 	 Loss BC 0.474207
Train Epoch: 170 	Loss: 0.6187198758 	 Loss BC 0.432116


In [ ]:
z = output.detach().numpy()
x = np.linspace(-1.,1.,100 )
y = np.linspace(-1.,1.,100)
x, y = np.meshgrid(x, y)
x = np.reshape(x, (np.size(x[:]),1))
y = np.reshape(y, (np.size(y[:]),1)) 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x,y,z)
plt.show()